In [1]:
from google.colab import drive
import os
import sys
import shutil
import subprocess

drive.mount('/content/drive')

PROJECT_DIR = '/content/drive/MyDrive/SVGEditor'
CODE_PATH = f"{PROJECT_DIR}/code"
DATA_PATH = f"{PROJECT_DIR}/data"
TEST_PATH = f"{PROJECT_DIR}/test"
MODEL_PATH = f"{PROJECT_DIR}/models"

TARGET_NAMES = ["apple", "daisy"]
TEST_CASE_PATH = f"{TEST_PATH}/testCase"
ORIGINAL_SVGS_PATH = f"{TEST_CASE_PATH}/OriginalSVGs"
RESULTS_PATH = f"{TEST_CASE_PATH}/Results"
COMPARISON_PATH = f"{TEST_CASE_PATH}/Comparison"
INTERMEDIATE_FILES_PATH = f"{TEST_CASE_PATH}/IntermediateFiles"

Mounted at /content/drive


In [2]:
# 1. Clean existing code directory and clone fresh repository
if os.path.exists(CODE_PATH):
    shutil.rmtree(CODE_PATH)

os.chdir(PROJECT_DIR)
result = subprocess.run(['git', 'clone', 'https://github.com/huanbasara/SVGEditor.git', 'code'],
                       capture_output=True, text=True)
print(f"Code repository {'successfully' if result.returncode == 0 else 'failed'} to {CODE_PATH}{f', {result.stderr}' if result.returncode != 0 else ''}")

# 2. Display latest commit information
os.chdir(CODE_PATH)
commit_info = subprocess.run(['git', 'log', '-1', '--pretty=format:%H|%ci|%s'],
                           capture_output=True, text=True)

if commit_info.returncode == 0:
    hash_code, commit_time, commit_msg = commit_info.stdout.strip().split('|', 2)
    print(f"Latest commit:\n   Hash: {hash_code[:8]}\n   Time: {commit_time}\n   Message: {commit_msg}")

# 3. Add code path to Python sys.path so we can import our modules
if CODE_PATH not in sys.path:
    sys.path.insert(0, CODE_PATH)
    print(f"✅ Added {CODE_PATH} to Python path")

# 4. Clear custom modules from cache
modules_to_clear = [
    'sam_processor',  # 我们创建的SAM处理模块
    'svglib',         # svglib包
    'utils'           # utils模块
]

for base in modules_to_clear:
    to_remove = [m for m in sys.modules if m.startswith(base)]
    for m in to_remove:
        del sys.modules[m]

print("✅ Modules reloaded!")

Code repository successfully to /content/drive/MyDrive/SVGEditor/code
Latest commit:
   Hash: 065d473e
   Time: 2025-09-22 10:07:35 +1200
   Message: update@2025-09-22 10:07:35
✅ Added /content/drive/MyDrive/SVGEditor/code to Python path
✅ Modules reloaded!


In [3]:
%pip install diffusers accelerate safetensors transformers huggingface_hub segment-anything opencv-python pillow matplotlib scikit-image scikit-learn cairosvg moviepy shapely networkx lxml cairosvg moviepy shapely networkx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00


In [4]:
# ========================================
# IP-Adapter 批量图像编辑系统 - 参数组合测试版
# ========================================

import torch
import gc
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
import matplotlib.pyplot as plt
import os
from utils import read_svg_file, svg_code_to_pil_image, get_prompt, get_edit_prompts

# 参数组合配置
PARAM_COMBINATIONS = [
    {"strength": 0.3, "guidance_scale": 7.0, "ip_adapter_scale": 0.8},   # 原始参数
    {"strength": 0.5, "guidance_scale": 10.0, "ip_adapter_scale": 0.3},  # 增强编辑
    {"strength": 0.6, "guidance_scale": 12.0, "ip_adapter_scale": 0.2},  # 强编辑
    {"strength": 0.4, "guidance_scale": 8.0, "ip_adapter_scale": 0.5},   # 平衡版本
    {"strength": 0.2, "guidance_scale": 6.0, "ip_adapter_scale": 0.9},   # 保守版本
]

def load_diffusion_model():
    """加载并返回配置好的diffusion模型"""
    print("🔄 Loading SD 1.5 base model...")
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        f"{MODEL_PATH}/stable-diffusion-v1-5",
        torch_dtype=torch.float16,
        safety_checker=None,
        requires_safety_checker=False,
        low_cpu_mem_usage=True
    ).to("cuda")

    print("🔄 Loading IP-Adapter...")
    pipe.load_ip_adapter(
        f"{MODEL_PATH}/ip-adapter",
        subfolder="models",
        weight_name="ip-adapter_sd15.bin"
    )

    pipe.enable_vae_slicing()
    pipe.enable_model_cpu_offload()

    print("✅ 模型加载完成")
    return pipe

def process_target_with_prompt(pipe, target_name, edit_prompt, prompt_index, param_index, params):
    """
    处理单个目标图像

    Args:
        pipe: 预加载的diffusion模型
        target_name: 目标名称 (如 "apple", "daisy")
        edit_prompt: 编辑提示词
        prompt_index: 提示词编号
        param_index: 参数组合编号
        params: 参数字典
    """
    print(f"\n=== 处理 {target_name} - prompt {prompt_index} - params {param_index} ===")

    # 1. 动态生成路径
    input_svg_path = f"{ORIGINAL_SVGS_PATH}/{target_name}.svg"
    intermediate_dir = f"{INTERMEDIATE_FILES_PATH}/{target_name}"
    os.makedirs(intermediate_dir, exist_ok=True)

    # 2. SVG转PIL Image
    print("🔄 Converting SVG to PIL Image...")
    svg_code = read_svg_file(input_svg_path)
    pil_image = svg_code_to_pil_image(svg_code, width=512, height=512, dpi=300)

    # 保存原始PNG到中间文件（只保存一次）
    original_png_path = os.path.join(intermediate_dir, f"{target_name}_original.png")
    if not os.path.exists(original_png_path):
        pil_image.save(original_png_path)

    # 3. 调整图像尺寸
    pil_image_256 = pil_image.resize((256, 256), Image.Resampling.LANCZOS)
    style_reference = pil_image_256

    # 4. 构建完整prompt
    full_prompt = f"{edit_prompt}; keep flat colors; anime/cartoon style; maintain original color palette; preserve white background"

    negative_prompt = "colored background, photorealistic, detailed textures, shading, gradients, shadows, noise, blur, 3d render, realistic, abstract, pattern, complex colors, dark background"

    print(f"编辑提示: {edit_prompt}")
    print(f"参数: S={params['strength']} G={params['guidance_scale']} IP={params['ip_adapter_scale']}")

    # 5. IP-Adapter推理
    print("🎨 开始 IP-Adapter 推理...")
    result = pipe(
        prompt=full_prompt,
        negative_prompt=negative_prompt,
        image=pil_image_256,
        ip_adapter_image=style_reference,
        strength=params['strength'],
        guidance_scale=params['guidance_scale'],
        ip_adapter_scale=params['ip_adapter_scale'],
        num_inference_steps=25,
        height=256,
        width=256,
        num_images_per_prompt=1,
        generator=torch.Generator().manual_seed(42)
    ).images[0]

    # 6. 生成文件名（包含参数信息）
    file_suffix = f"p{prompt_index}_param{param_index}"

    # 保存中间结果
    edit_filename = f"{target_name}_edit_{file_suffix}.png"
    edit_path = os.path.join(intermediate_dir, edit_filename)
    result.save(edit_path)

    # 7. 创建并保存对比图（包含参数信息）
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    axes[0].imshow(pil_image_256)
    axes[0].set_title("Original")
    axes[0].axis('off')

    axes[1].imshow(style_reference)
    axes[1].set_title("Style Reference")
    axes[1].axis('off')

    axes[2].imshow(result)
    axes[2].set_title(f"Edit {prompt_index} - Param {param_index}")
    axes[2].axis('off')

    # 添加edit prompt和参数信息
    param_text = f"S={params['strength']}, G={params['guidance_scale']}, IP={params['ip_adapter_scale']}"
    fig.text(0.5, 0.08, f"Edit Prompt: {edit_prompt}",
             fontsize=12, ha='center', wrap=True,
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray", alpha=0.8))
    fig.text(0.5, 0.02, f"Parameters: {param_text}",
             fontsize=10, ha='center',
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.20)  # 给底部文字留更多空间

    # 保存对比图到comparison目录
    comparison_filename = f"{target_name}_{file_suffix}.png"
    comparison_path = os.path.join(COMPARISON_PATH, comparison_filename)
    plt.savefig(comparison_path, bbox_inches='tight', dpi=150)
    plt.show()

    # 8. 保存最终结果到results目录
    final_result_path = os.path.join(RESULTS_PATH, f"{target_name}_{file_suffix}.png")
    result.save(final_result_path)

    print(f"✅ 处理完成!")
    print(f"  - 中间文件: {edit_path}")
    print(f"  - 对比图: {comparison_path}")
    print(f"  - 最终结果: {final_result_path}")

    return result

def batch_process_targets_with_params(target_names, get_edit_prompts_func):
    """
    批量处理多个目标，测试不同参数组合

    Args:
        target_names: 目标名称列表
        get_edit_prompts_func: 获取编辑提示的函数
    """
    # 创建必要目录
    os.makedirs(COMPARISON_PATH, exist_ok=True)
    os.makedirs(RESULTS_PATH, exist_ok=True)

    # 一次性加载模型
    pipe = load_diffusion_model()

    try:
        for target_name in target_names:
            edit_prompts = get_edit_prompts_func(target_name)

            for prompt_idx, edit_prompt in enumerate(edit_prompts):
                for param_idx, params in enumerate(PARAM_COMBINATIONS):
                    process_target_with_prompt(
                        pipe, target_name, edit_prompt,
                        prompt_idx, param_idx, params
                    )

                    # 清理GPU缓存
                    torch.cuda.empty_cache()
                    gc.collect()

    finally:
        # 清理模型
        del pipe
        torch.cuda.empty_cache()
        gc.collect()
        print("✅ 所有任务完成，模型已清理")

def get_merged_prompts(target):
    # 获取原始prompt
    original_prompt = get_prompt(target)

    # 编辑提示词
    edit_prompts_raw = {
        "apple": ["remove the leaves", "add more leaves"],
        "daisy": ["make the center of the flower smaller"]
    }

    # 拼接原始prompt和编辑prompt
    raw_prompts = edit_prompts_raw.get(target, ["default edit"])
    combined_prompts = []

    for edit_prompt in raw_prompts:
        combined_prompt = f"Original image: {original_prompt}. Edit instruction: {edit_prompt}. Keep the same art style and color scheme."
        combined_prompts.append(combined_prompt)

    return combined_prompts

def force_cuda_reset():
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.synchronize()

# 显示参数组合信息
print("=== 参数组合测试 ===")
for i, params in enumerate(PARAM_COMBINATIONS):
    print(f"组合 {i}: S={params['strength']}, G={params['guidance_scale']}, IP={params['ip_adapter_scale']}")

force_cuda_reset()

# 批量处理
batch_process_targets_with_params(TARGET_NAMES, get_merged_prompts)

Output hidden; open in https://colab.research.google.com to view.